# 整理传入CodonW文件

In [ ]:
#-*-coding:utf-8-*-
import json
import numpy as np
import random
from tqdm import tqdm
import math 
import re
import pandas as pd
import os 
import datetime
from multiprocessing import Pool
from matplotlib import pyplot as plt

In [12]:
###修改路径
###计算所有参数取平均值

os.chdir('/home/dell-none/Desktop/0426jiqun')
df=pd.read_csv('slope_all_SSR.csv')

In [13]:
###计算所有参与分析的序列的密码子指数
def get_dic(filename):

    dic = {}

    with open(filename, 'r')as f1:

        lines = f1.readlines()

    for line in lines:

        if '>' in line:

            epi = line.rstrip('\n')

            dic[epi] = ''

        else:

            dic[epi] += line.strip('\n')

    return dic

dic1=get_dic('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/seq.fasta')

from collections import Counter


###CodonW分析序列不能含模糊碱基，先填补模糊碱基
def highfre_site(seq,n):
    
    ### 获取最高频碱基
    array=np.array(seq)
    s=Counter(array).most_common(1)[0][0]###得到碱基个数最多的一个碱基
    if s!="N":
        pass
    
    else:
        if n%4==1:
            s="A"
        if n%4==2:
            s="T"
        if n%4==3:
            s="G"
        if n%4==0:
            s="C"
    return s
high_site=[]

lenth=max([len(v) for v in dic1.values()])
print(lenth)
n=0
for i in range(lenth):
    
    n+=1
    ### 获取每个位点构成序列
    seq=[]
    for k,v in dic1.items():
        seq.append(v[i])
    # print(seq)
    s=highfre_site(seq,n)
    high_site.append(s)
# print(high_site)   
    
ff_dic={}
for k,v in dic1.items():
    ###更新序列，填补gap
    v1=''
    for i in range(lenth):
        if v[i]=='N':
            v1+=high_site[i]
        else:
            v1+=v[i]
    ff_dic[k]=v1

###序列滑窗切片
def get_seq(s1,s2,n):
    ss1=dic1[s1][n:n+600]
    ss2=dic1[s2][n:n+600]
    return ss1,ss2

###CodonW分析序列需以起始密码子开始，以终止密码子结束，故，对每个滑窗片段添加起始和终止密码子
spl={}
for i in tqdm(list(df['Sequence'])):
    seq1=i.split(',')[0]
    seq2=i.split(',')[1].split('(')[0]
    num_pos=int(i.split(',')[1].split('(')[1].replace(')',''))
    
    n1,n2=seq1+'_'+str(num_pos),seq2+'_'+str(num_pos)
    v1,v2=get_seq(seq1,seq2,num_pos)
    
    if n1 not in spl.keys():
        spl[n1]='ATG'+v1+'TAA'
    
    if n2 not in spl.keys():
        spl[n2]='ATG'+v2+'TAA'
###写入fasta文件，终端传入CodonW，得到每条序列分析的密码子指数文件      
def write_file(file,dict):
    
    with open(file,'w')as f2:
        for k1,v1 in dict.items():
            f2.write(k1+'\n'+v1+'\n')
write_file('codonw_seq.fasta',spl)

# CodonW结果提取

In [ ]:
codonw=pd.read_csv('codonw_out.csv')
codonw['seq']=[i.rstrip() for i in list(codonw['seq'])]
add=df[['Sequence','Time interval','Synonymous_substitution_rate']]



#### 并行匹配codonw结果
def get_pos(i):
    return i.split('(')[1].replace(')','')
    
    
    
p1 = re.compile(r'[(](.*?)[)]', re.S)


def run(add,df):
    CBI_,ENC_,Gravy_,T3,A3,G3,C3,GC3,GC=[],[],[],[],[],[],[],[],[]
    for i in tqdm(list(add['Sequence'])):
        
        seq1=i.split(',')[0]+'_'+re.findall(p1, i)[0]
        seq2=i.split(',')[1].split('(')[0]+'_'+re.findall(p1, i)[0]
        
        n1=df[df['seq']==seq1];n2=df[df['seq']==seq2]
        
        T3.append(abs(n1.iloc[0,0]-n2.iloc[0,0]))
        C3.append(abs(n1.iloc[0,1]-n2.iloc[0,1]))
        A3.append(abs(n1.iloc[0,2]-n2.iloc[0,2]))
        G3.append(abs(n1.iloc[0,3]-n2.iloc[0,3]))
        CBI_.append(abs(n1.iloc[0,4]-n2.iloc[0,4]))
        GC3.append(abs(n1.iloc[0,5]-n2.iloc[0,5]))
        GC.append(abs(n1.iloc[0,6]-n2.iloc[0,6]))
        Gravy_.append(abs(n1.iloc[0,8]-n2.iloc[0,8]))
        ENC_.append(abs(n1.iloc[0,10]-n2.iloc[0,10]))
    
    
    add['T3']=T3;add['C3']=C3;add['A3']=A3;add['G3']=G3;
    add['CBI']=CBI_;add['GC3']=GC3;add['GC']=GC;add['Gravy']=Gravy_;
    add['ENC']=ENC_

    return add

num=[i for i in range(0,1097,100)]
num.append(1097)
para=[]
for i in range(len(num)-1):
    splt_df=add.iloc[num[i]:num[i+1],:]
    para.append((splt_df,df))
# del add

def cal_dist_mat(para):
    with Pool(12) as pool:
        res=pool.starmap(run,para)
        pool.close()
        pool.join()
    return list(res)
re_lt=cal_dist_mat(para)

for i in range(len(re_lt)):
    re_lt[i].to_csv('codonw_match_result%s.csv'%i,sep=',',index=False)

# 添加CpG,UpA

In [ ]:
###读取fasta文件
def get_dic(filename):
    dic = {}
    with open(filename, 'r')as f1:
        lines = f1.readlines()
    for line in lines:
        if '>' in line:
            epi = line.rstrip('\n')

            dic[epi] = ''
        else:
            dic[epi] += line.strip('\n')
    return dic

dic=get_dic('codonw_seq.fasta')

###此处传入df为两两序列节点对原始计算的表格，需要先计算出序列对之间的差值绝对值，再求位点平均
add=df[['Sequence','Time interval','Synonymous_substitution_rate']]

###函数计算cpg，upa含量
def cpg_upa(fa_dic):
    result=[]
    for k,v in tqdm(fa_dic.items()):
        cg_num=0;ua_num=0;cg_mul=v.count('C')*v.count('G');ua_mul=v.count('T')*v.count('A')
        for i in range(len(v)):
            if v[i:i+2]=='CG':
                cg_num+=1
            if v[i:i+2]=='TA':
                ua_num+=1
        cg=round(cg_num/cg_mul,4);ua=round(ua_num/ua_mul,4)
        result.append([k,cg,ua])
    
#     df=pd.DataFrame(result,columns=['Sequence','CpG_enrichment','UpA enrichment'])
    return result

result=cpg_upa(dic)
ter={}
for i in result:
    ter[i[0]]=[i[1],i[2]]
###匹配cpg，upa   
p1 = re.compile(r'[(](.*?)[)]', re.S)
def run(add,ter):
    
    cpg=[];upa=[]
    
    for i in tqdm(list(add['Sequence'])):
        
        seq1=i.split(',')[0]+'_'+re.findall(p1, i)[0]
        seq2=i.split(',')[1].split('(')[0]+'_'+re.findall(p1, i)[0]
        
        n1,n2=ter[seq1],ter[seq2]
        
        cpg.append(abs(n1[0]-n2[0]))
        upa.append(abs(n1[1]-n2[1]))
    
    add['CpG']=cpg;add['UpA']=upa
    
    return add

add2=run(add,ter)

####对所有
add2=add2.groupby(['Position']).mean()
add2.to_csv('add_cpgupa_codonw.csv',index=False,sep=',')

# RNA二级结构能量计算

In [ ]:
#-*-coding:utf-8-*-

import json

import numpy as np

import random

from tqdm import tqdm

import math 

import re

import pandas as pd

import pickle

import pickle5

import networkx as nx

import os 

import datetime

from multiprocessing import Pool
from matplotlib import pyplot as plt

In [ ]:
seq_single=get_dic('codonw_seq.fasta')

only_n={}
for k,v in seq_single.items():
    v1=v[3:-3]

    
    
    only_n[k]=v1
ew={}

for k,v in seq_single.items():
    
    if v not in ew.keys():
        
        ew[k]=[]

for k1 in tqdm(ew.keys()):
    for k,v in dic1.items():
        if v==k1:
            ew[k1].append(k)
            
####写程序重复调用，这部分计算较慢，可用集群计算
####先将codonw_seq.fasta文件拆分成若干个文件，这里拆分成972个，并行计算
def fun(file,path):
    os.chdir('/home/dell-none/Downloads/RNAstructureLinuxTextInterfaces64bit/RNAstructure/exe/')
    os.system('./Fold %s %s --loop 30 --maximum 20 --percent 10 --temperature 310.15 --window 3'%(file,path))
    

def main():
        pool = Pool(12)
        n=0
        for n in range(0,972):
            filename='/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt/RNA_secondary/orign_fasta/%s.fasta'%n
            filepath='/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt/RNA_secondary/out_file/test_out_%s.ct'%n
            pool.apply_async(fun, args=(filename,filepath))

        pool.close()
        pool.join()
if __name__ == '__main__':
    main()

###按位置计算energy
mark=[];slt={}
for k,v in tqdm(only_n.items()):
    
    if k.split('_')[3] not in mark:
        mark.append(k.split('_')[3])
        slt[k]=v
#     break
    

In [ ]:
# os.chdir('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt')
df_all=pd.read_csv('add_cpgupa_codonw.csv')

os.chdir('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt/RNA_secondary/out_file')
energy=[]
for i in tqdm(range(1,971)):
    with open('test_out_%s.ct'%i,'r') as f1:
        lines=f1.readlines()[0].strip().split('\t')[0]
    energy.append(lines) 
    
energy_pos={}
for i in energy:
    k=int(i.split('  ')[2].split('_')[3])
    energy_pos[k]=float(i.split('  ')[1].split('=')[1].strip())
    
energy_df=pd.DataFrame.from_dict(energy_pos, orient='index',columns=['energy'])
energy_df = energy_df.reset_index().rename(columns = {'index':'id'})

df_all['Position']=[int(i.split("(")[1].replace(')','')) for i in list(df_all['Sequence'])]
df_all['RNA_scondary']=[energy_pos[i] for i in df_all['Position']]


#  添加aaindex

In [ ]:

def get_aaindex(filename):
    IndexDic = {}
    with open(filename) as f1:
        lines = f1.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('D'):
            index = lines[i].rstrip('\n').split('(')[0].replace('D ','').rstrip()

            IndexDic[index] = ''
        if lines[i].startswith('I'):

            num=[]
            AA=re.split(r"[ ]+", lines[i].rstrip('\n'))[1:]

            first=re.split(r"[ ]+", lines[i+1].rstrip('\n'))[1:]

            second=re.split(r"[ ]+", lines[i+2].rstrip('\n'))[1:]

            for  i in range(len(AA)):
                num.append((AA[i].split('/')[0],first[i]))
                num.append((AA[i].split('/')[1],second[i]))


            IndexDic[index]=num
                         
                
    return IndexDic

IndexDic=get_aaindex('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt/aaindex')


table = {
'UUU':'F','CUU':'L','AUU':'I','GUU':'V',
'UUC':'F','CUC':'L','AUC':'I','GUC':'V',
'UUA':'L','CUA':'L','AUA':'I','GUA':'V',
'UUG':'L','CUG':'L','AUG':'M','GUG':'V',
'UCU':'S','CCU':'P','ACU':'T','GCU':'A',
'UCC':'S','CCC':'P','ACC':'T','GCC':'A',
'UCA':'S','CCA':'P','ACA':'T','GCA':'A',
'UCG':'S','CCG':'P','ACG':'T','GCG':'A',
'UAU':'Y','CAU':'H','AAU':'N','GAU':'D',
'UAC':'Y','CAC':'H','AAC':'N','GAC':'D',
'UAA':'-','CAA':'Q','AAA':'K','GAA':'E',
'UAG':'-','CAG':'Q','AAG':'K','GAG':'E',
'UGU':'C','CGU':'R','AGU':'S','GGU':'G',
'UGC':'C','CGC':'R','AGC':'S','GGC':'G',
'UGA':'-','CGA':'R','AGA':'R','GGA':'G',
'UGG':'W','CGG':'R','AGG':'R','GGG':'G'
}
codon_table={}
for k,v in table.items():
    k=k.replace('U','T')
    codon_table[k]=v
    
def re_pro(dic):
    aa_dic={}
    for k,v in tqdm(dic.items()):
        aa_seq=''
        for i in range(0,len(v),3):
            aa_seq+=codon_table[v[i:i+3]]
        aa_dic[k]=aa_seq
    return aa_dic

def get_value(aa,v):
    for i in v:
        if aa==i[0]:
#             print(float(i[1]))
            return float(i[1])

def get_index(aaseq):
    list_=[]
    for k,v in IndexDic.items():
        result=0
        for i in aaseq:
            try:
                result+=get_value(i,v)
            except:
                result+=0
        list_.append(result)
    return list_

aa_dic=re_pro(only_n)

new_index={}
for name,aaseq in tqdm(aa_dic.items()):
    new_index[name]=get_index(aaseq)
    
aa1,aa2=[],[]
for i in tqdm(list(df_all['Sequence'])):
    kk='_'+i.split(',')[1].split('(')[1].replace(')','')
    seq1=i.split(',')[1].split('(')[0]+kk
    seq2=i.split(',')[0]+'_'+i.split(',')[1].split('(')[1].replace(')','')
    aaindex1,aaindex2=abs(new_index[seq1][0]-new_index[seq2][0]),abs(new_index[seq1][1]-new_index[seq2][1])
    aa1.append(aaindex1);aa2.append(aaindex2)
    
df_all['Normalized van der Waals volume']=aa1
df_all['Relative mutability']=aa2

df_all.drop('Position',axis=1,inplace=True)

df_all.to_csv('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/fas/axt/parameter.csv',sep=',',index=False)

# 加入motif是否存在的信息

In [ ]:
###motif_fimo为03中用FIMO扫描motif所对应的位置得到motif_low_match.tsv文件，对motif和position去冗余之后的dataframe，即为para_pos_fimo.tsv
####motif信息改成每个参数格式
motif_fimo=pd.read_csv('para_pos_fimo.tsv',sep='\t')

motif_fimo['sequence_name']=['>'+i for i in list(motif_fimo['sequence_name'])]
motif_info=pd.DataFrame(columns=sorted(list(set(list(motif_fimo['motif_id'])))))

data=pd.read_csv('parameter.csv')

data_=data[['Sequence', 'Synonymous_substitution_rate', 'T3', 'C3', 'A3', 'G3',
       'CBI', 'GC3', 'GC', 'Gravy', 'ENC', 'CpG', 'UpA', 'nrs', 'w',
       'RNA_scondary', 'Normalized van der Waals volume',
       'Relative mutability','position']]


motif_info['Sequence']=data_['Sequence']
motif_info['Position']=data_['position']
dic_inter=dict(enumerate(motif_info.columns))
dic_new = dict(zip(dic_inter.values(), dic_inter.keys()))
dic_new


alst=list(motif_info['Sequence']);blst=list(motif_info['Position'])
###一对节点中均存在标注为2，只有一条序列存在标注为1，都不存在该motif标注为0
for i in tqdm(range(len(alst))):
    seq1=alst[i].split(',')[0]
    seq2=alst[i].split(',')[1]
    name1,name2=seq1+'_'+str(blst[i]),seq2+'_'+str(blst[i])
    a=list(motif_fimo[motif_fimo['sequence_name']==name1]['motif_id'])
    b=list(motif_fimo[motif_fimo['sequence_name']==name2]['motif_id'])
#     print(a)
    
    if len(a)!=0 or len(b)!=0:
    
        for j in a:
            if j in  b:
                motif_info.iloc[i,dic_new[j]]=2
            else:
                motif_info.iloc[i,dic_new[j]]=1
        for q in b:
            if q in a:
                motif_info.iloc[i,dic_new[q]]=2
            else:
                motif_info.iloc[i,dic_new[q]]=1

motif_info.head()
motif_info = motif_info.fillna(0)
motif_info.to_csv('motif_para.csv',index=False,sep=',')        

# 建lightGBM模型

In [ ]:
data_parameter_model=pd.concat([data_,motif_info.iloc[:,:10]],axis=1)

from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

data_parameter_model.columns

import matplotlib as mpl
import matplotlib.pyplot as plt

# 设置西文字体为新罗马字体
from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',  # 设置字体类型
#     "font.size": 80,
#     "mathtext.fontset":'stix',
}
rcParams.update(config)


data2=data_parameter_model

plt.figure(figsize=(20,16))
cor = data2.iloc[:,1:14].corr(method='spearman')
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
# plt.show()
plt.xticks(fontsize=20,rotation=90)
plt.yticks(fontsize=20)
plt.savefig('heatmap.pdf',format='pdf',bbox_inches='tight',dpi=300,facecolor='white')
# 筛选出于因变量之间的相关性
cor_target = abs(cor['Synonymous_substitution_rate'])
# 挑选于大于0.4的相关性系数
relevant_features = cor_target[cor_target>0.01]
relevant_features

###依据相关性删除一些特征
data2.drop(['Normalized van der Waals volume','G3','Gravy','Relative mutability','position'],axis=1,inplace=True)


data2.columns=['Sequence', 'Synonymous_substitution_rate', 'T3', 'C3', 'A3', 'CBI',
       'GC3', 'GC', 'ENC', 'CpG', 'UpA', 'NER', 'w', 'RNA_scondary',
       '1-TAGCTGTCT', '10-CAATAGTCT', '2-AAAACTA', '3-CATTAACAACCACTG',
       '4-ATGAAAATTTT', '5-AMAMKTKT', '6-ATGTAATCATCA', '7-TCAACAAA',
       '8-ATGTCTTCA', '9-GGTTTACT']

###划分训练集测试集
X_train2, X_test2, y_train2, y_test2 = train_test_split(data2.iloc[:,2:],data2.iloc[:,1] , test_size=0.2)
X2=data2.iloc[:,1:];y2=data2.iloc[:,0]


###训练模型
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

###默认参数
params = {
    'num_leaves': [7, 14, 21, 28, 31, 50],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_depth': [-1, 3, 5],
    'n_estimators': [50, 100, 200, 500],
    'reg_lambda':[1e-2,0.5,1]
}

###交叉验证
grid = GridSearchCV(lgb.LGBMRegressor(random_state=1001,objective='regression'),params,scoring='neg_mean_squared_error',cv=10)
grid.fit(X_train2, y_train2)


###最好模型的性能参数
lgbm_tuned3=grid.best_estimator_

r2_tuned2=grid.best_params_

###训练集结果
lgbm_tuned3
r2_train2=r2_score(y_train2,grid.predict(X_train2))
mse_train2=mean_squared_error(y_train2,grid.predict(X_train2))
###测试集结果
r2_test2=r2_score(y_test2,lgbm_tuned3.predict(X_test2))
mse_test2=mean_squared_error(y_test2, lgbm_tuned3.predict(X_test2))
print(r2_train2,r2_test2)
print(mse_train2,mse_test2)

from sklearn import  metrics
print(metrics.median_absolute_error(y_test2, lgbm_tuned3.predict(X_test2)))
print(metrics.median_absolute_error(y_train2,lgbm_tuned3.predict(X_train2)))

###计算调整R2，数字为样本数量和特征数量
tr_adj_r2=1-(1-r2_train2)*76903/(76904-22-1)
te_adj_r2=1-(1-r2_test2)*19226/(19227-22-1)
print(tr_adj_r2,te_adj_r2)

In [ ]:
###SHAP可解释特征value，及特征重要性画图

import matplotlib as mpl
import matplotlib.pyplot as plt

# 设置西文字体为新罗马字体
from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',  # 设置字体类型
#     "font.size": 80,
#     "mathtext.fontset":'stix',
}
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams.update(config)



import shap
lgbm_tuned3.fit(X_train2, y_train2)

explainer = shap.TreeExplainer(lgbm_tuned3)
shap_values = explainer.shap_values(data2.iloc[:,2:])

from matplotlib import pyplot as plt

shap.summary_plot(shap_values, data2.iloc[:,2:],show=False)

plt.savefig('SHAP_value_final_version.pdf',format='pdf',bbox_inches='tight',dpi=300,facecolor='white')

In [ ]:
###计算特征贡献程度
shapva=pd.DataFrame(shap_values,columns=list(data2.columns)[2:])
mean_shap_value=[]
for i in range(len(shapva.columns)):
    mean_shap_value.append(np.mean([abs(j) for j in list(shapva.iloc[:,i])]))

con_per=[i/np.sum(mean_shap_value)*100 for i in mean_shap_value]
character_contri=pd.DataFrame([list(shapva.columns),mean_shap_value,con_per]).T
character_contri.columns=['x','SHAP_value','percentage']

character_contri.to_csv("feature_contribution.csv",index=False,sep=',')